In [1]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap

# from rascaline.utils import clebsch_gordan
import clebsch_gordan

# Test system 1

In [2]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# Define target lambda channels
lambdas = np.array([0, 2])

In [6]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

22.5 s ± 925 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

12.9 s ± 1.92 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [8]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body

TensorMap with 8 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         3             1                   0                  1
         3            -1                   2                  1
                                    ...
         3             1                   2                  8
         3            -1                   0                  8

In [ ]:
# selected_samples = None
# species_neighbors = [1, 8, 6]

# calculator = rascaline.SphericalExpansion(**rascal_hypers)
# nu1 = calculator.compute(frames, selected_samples=selected_samples)

# # Move the "species_neighbor" key to the properties. If species_neighbors is
# # passed as a list of int, sparsity can be created in the properties for
# # these species.
# if species_neighbors is None:
#     keys_to_move = "species_neighbor"
# else:
#     keys_to_move = Labels(
#         names=["species_neighbor"],
#         values=np.array(species_neighbors).reshape(-1, 1),
#     )
# nu1 = nu1.keys_to_properties(keys_to_move=keys_to_move)
# nu1 = clebsch_gordan._add_nu_sigma_to_key_names(nu1)